In [1]:
# Copyright (C) 2022, Advanced Micro Devices, Inc.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# * Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.
#
# * Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.
#
# * Neither the name of FINN nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

import pytest
import os

import numpy as np
from onnx import TensorProto, helper
from qonnx.util.basic import (
    qonnx_make_model,
    gen_finn_dt_tensor
)
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from qonnx.transformation.general import GiveUniqueNodeNames
import finn.core.onnx_exec as oxe
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from qonnx.transformation.general import ApplyConfig
import finn.transformation.fpgadataflow.specialize_to_rtl_layers as to_rtl
from qonnx.custom_op.general.im2col import compute_conv_output_dim
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.general import GiveReadableTensorNames

#import qonnx.core.data_layout as DataLayout

build_dir = os.environ["FINN_BUILD_DIR"]

def make_single_dw_conv_modelwrapper(conv_config, idt, wdt):
    kernel_size, in_feature_dim, in_chn = conv_config
    stride = 1
    pad = 0

    out_feature_dim = compute_conv_output_dim(in_feature_dim, kernel_size, stride, pad)
    group = out_chn = in_chn
    
    conv_param_shape = [out_chn, 1, kernel_size, kernel_size]
    input_shape = [1, in_chn, in_feature_dim, in_feature_dim]
    output_shape = [1, out_chn, out_feature_dim, out_feature_dim]

    conv_config = {}
    conv_config["dilations"] = [1, 1]
    conv_config["group"] = group
    conv_config["kernel_shape"] = [kernel_size, kernel_size]
    conv_config["pads"] = [pad, pad, pad, pad]
    conv_config["strides"] = [stride, stride]

    ifm = helper.make_tensor_value_info("ifm", TensorProto.FLOAT, input_shape)
    ofm = helper.make_tensor_value_info("ofm", TensorProto.FLOAT, output_shape)
    weights = [helper.make_tensor_value_info("weights", TensorProto.FLOAT, conv_param_shape)]

    modelproto = qonnx_make_model(
        helper.make_graph(
            name="conv_test",
            inputs=[ifm],
            outputs=[ofm],
            value_info=weights,
            nodes=[helper.make_node("Conv", ["ifm", "weights"], ["ofm"], **conv_config)],
        )
    )

    model = ModelWrapper(modelproto)
    model.set_tensor_datatype("ifm", idt)
    model.set_tensor_datatype("weights", wdt)
    model.set_initializer("weights", gen_finn_dt_tensor(wdt, conv_param_shape))

    model = model.transform(InferShapes())
    model = model.transform(InferDataTypes())

    return model

def prepare_inputs(input_tensor):
    return {"inp": input_tensor}

def test_fpgadataflow_vvau_rtl(kernel_size, in_feature_dim, in_chn, idt, wdt, part, segmentlen, pe, simd):
    # Create depthwise-separable convolution
    conv_config = (kernel_size, in_feature_dim, in_chn)
    model = make_single_dw_conv_modelwrapper(conv_config, idt, wdt)
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model.save(build_dir+"/dw_conv.onnx")

    # Obtain golden reference output
    golden_in = gen_finn_dt_tensor(model.get_tensor_datatype("global_in"), model.get_tensor_shape("global_out"))
    input_dict = prepare_inputs(golden_in)
    golden_out = oxe.execute_onnx(model, input_dict)

    # Convert to HLS custom-op first
    model = model.transform(LowerConvsToMatMul())
    model = model.transform(to_hls.InferConvInpGen(use_rtl_variant=True))
    model = model.transform(to_hls.InferVectorVectorActivation())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model.save(build_dir+"/hls_vvau.onnx")

    # Apply folding (i.e. specify to use DSPs)
    folding_config = {
        "Defaults": {},
        "ConvolutionInputGenerator_rtl_0": {
            "SIMD" : simd
        },
        "VectorVectorActivation_0": {
            "PE" : pe,
            "SIMD" : simd,
            "mem_mode" : "decoupled",
            "ram_style" : "auto",
            "resType" : "dsp",
            "preferred_backend" : "rtl"
        }
    }
    model = model.transform(ApplyConfig(folding_config))
    model.save(build_dir+"/hls_vvau_folded.onnx")

    # Obtain second reference from HLS-based VVAU layer
    model = model.transform(SetExecMode("rtlsim"))
    #model = model.transform(PrepareIP(part, 5))
    #model = model.transform(HLSSynthIP())
    #model = model.transform(PrepareRTLSim())
    #conv_hls_out = oxe.execute_onnx(model, input_dict, True) 
    #np.save(build_dir+"/hls_vvau_folded_output.npy", conv_hls_out)
    #assert (golden_out["global_out"] == conv_hls_out["global_out"]).all()

    # Apply convert-to-rtl step
    model = model.transform(to_rtl.InferRTLVectorVectorActivation())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model.save(build_dir+"/rtl_vvau.onnx")
    model = model.transform(PrepareIP(part, 5))
    model = model.transform(HLSSynthIP())
    model = model.transform(PrepareRTLSim())
    assert True
    #vvu_rtl_out = oxe.execute_onnx(model, input_dict, True)
    #np.save(build_dir+"/rtl_vvau_output.npy", vvu_rtl_out)


In [2]:
test_fpgadataflow_vvau_rtl(3, 10, 6, DataType["UINT8"], DataType["INT6"], "xcvm1802-vsvd1760-2MP-e-S", 1, 1, 3)

/scratch/mirzam/finn/deps/qonnx/src/qonnx/transformation/general.py:354: UserWarning: 
No HW configuration for nodes: Transpose_0, Transpose_1
  warnings.warn("\nNo HW configuration for nodes: " + ", ".join(missing_configurations))
/scratch/mirzam/finn/src/finn/transformation/fpgadataflow/hlssynth_ip.py:73: UserWarning: Using pre-existing IP for VectorVectorActivation_rtl_0
  warnings.warn("Using pre-existing IP for %s" % node.name)
/scratch/mirzam/finn/src/finn/transformation/fpgadataflow/hlssynth_ip.py:73: UserWarning: Using pre-existing IP for ConvolutionInputGenerator_rtl_0
  warnings.warn("Using pre-existing IP for %s" % node.name)
%Warning-WIDTH: /scratch/mirzam/build_files/code_gen_ipgen_ConvolutionInputGenerator_rtl_0_2rdz7x6t/swg_common.sv:69:78: Operator ASSIGN expects 4 bits on the Assign RHS, but Assign RHS's VARREF 'LOOP_H_ITERATIONS' generates 32 bits.
                                                                                                                       : 

make: Entering directory '/scratch/mirzam/build_files/pyverilator_ConvolutionInputGenerator_rtl_0_rmehhsuh'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /scratch/mirzam/build_files/pyverilator_ConvolutionInputGenerator_rtl_0_rmehhsuh/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std

# SIMD = 1

In [2]:
import numpy as np
import pickle

with open('hls_vvau_folded_output.pkl', 'rb') as f:
    loaded_dict_hls = pickle.load(f)

with open('rtl_vvau_output.pkl', 'rb') as f:
    loaded_dict_rtl = pickle.load(f)


In [3]:
loaded_dict_hls["Transpose_0_out0"]

array([[[[ -45.,  123.,   87.,  -56.],
         [-121.,  124.,   22., -121.],
         [ -92., -110.,  -94.,  102.],
         [  12.,   84.,  105.,    7.],
         [  97.,  103.,  115.,   29.]],

        [[ -20.,  -79.,  -79.,   77.],
         [ 121.,  122., -102.,  -61.],
         [ -57.,  -90.,  114., -110.],
         [  57.,   36., -114.,   73.],
         [-128.,  -38.,  -90.,    1.]],

        [[-119.,   -6.,  -75.,  -75.],
         [ -58.,   42.,   74.,   95.],
         [  11.,  -44.,   66.,  -66.],
         [ -75., -117.,   27.,   15.],
         [ -32.,   39.,  -68.,  -10.]],

        [[ -54.,   35.,  -30., -112.],
         [ -74.,   32.,   86.,   13.],
         [  41.,  127.,   56.,   41.],
         [  34.,  -51., -119.,   38.],
         [  82.,   44.,   46.,   34.]],

        [[ -48.,  -10.,   77.,   43.],
         [-113., -116.,  -87.,   -8.],
         [-108.,  106., -107.,  -10.],
         [  61.,  -97.,   71.,   43.],
         [ -50.,   42.,   51.,  -65.]]]], dtype=float32)

In [4]:
loaded_dict_hls["ConvolutionInputGenerator_rtl_0_out0"]

array([[[[ -45., -121.,  -92.,  -20.,  121.,  -57., -119.,  -58.,
            11.,  123.,  124., -110.,  -79.,  122.,  -90.,   -6.,
            42.,  -44.,   87.,   22.,  -94.,  -79., -102.,  114.,
           -75.,   74.,   66.,  -56., -121.,  102.,   77.,  -61.,
          -110.,  -75.,   95.,  -66.],
         [-121.,  -92.,   12.,  121.,  -57.,   57.,  -58.,   11.,
           -75.,  124., -110.,   84.,  122.,  -90.,   36.,   42.,
           -44., -117.,   22.,  -94.,  105., -102.,  114., -114.,
            74.,   66.,   27., -121.,  102.,    7.,  -61., -110.,
            73.,   95.,  -66.,   15.],
         [ -92.,   12.,   97.,  -57.,   57., -128.,   11.,  -75.,
           -32., -110.,   84.,  103.,  -90.,   36.,  -38.,  -44.,
          -117.,   39.,  -94.,  105.,  115.,  114., -114.,  -90.,
            66.,   27.,  -68.,  102.,    7.,   29., -110.,   73.,
             1.,  -66.,   15.,  -10.]],

        [[ -20.,  121.,  -57., -119.,  -58.,   11.,  -54.,  -74.,
            41.,  -79.,

# Some tests

In [1]:
import numpy as np
import pickle

with open('hls_vvau_folded_output.pkl', 'rb') as f:
    dict_hls = pickle.load(f)

with open('onnx_dws_conv.pkl', 'rb') as f:
    dict_onnx = pickle.load(f)

with open('stitched_ip_output.pkl', 'rb') as f:
    dict_stitched_ip = pickle.load(f)

with open('rtl_vvau_output.pkl', 'rb') as f:
    dict_rtl = pickle.load(f)

In [2]:
print(dict_onnx["global_out"].shape)
dict_onnx["global_out"]

(1, 4, 3, 3)


array([[[[ 4851., -5781.,  -466.],
         [ -374., -4851.,  -633.],
         [ 2689., -2184.,  -457.]],

        [[ 2931.,  2494.,  2266.],
         [ 2672.,  1388.,    24.],
         [ 3199.,  7602.,  5672.]],

        [[ -511.,  6304., -2546.],
         [ 1704., -5466.,  2307.],
         [ 5092.,  5691.,  1111.]],

        [[  644., -6511., -1961.],
         [ 4175., -1759., -4617.],
         [-2317.,  4556., -3143.]]]], dtype=float32)

In [3]:
print(dict_hls["global_out"].shape)
dict_hls["global_out"]

(1, 4, 3, 3)


array([[[[ 3326.,  -494., -2079.],
         [ 1161., -2468.,  1943.],
         [-2519.,  2378., -4647.]],

        [[ 3418.,  4054.,  5345.],
         [ 6199.,  2266., -4485.],
         [ 4412.,  5648.,  4910.]],

        [[-1952.,   764.,   283.],
         [ 2094.,  3534.,  8048.],
         [10425., -2818.,  9857.]],

        [[ 4084., -1630.,  3902.],
         [ 3294., -2200.,  3158.],
         [ 2638.,  1600.,  2399.]]]], dtype=float32)

In [4]:
print(dict_stitched_ip["global_out"].shape)
np.transpose(dict_stitched_ip["global_out"], (0,3,1,2))

(1, 3, 3, 4)


array([[[[ 4851., -5781.,  -466.],
         [ -374., -4851.,  -633.],
         [ 2689., -2184.,  -457.]],

        [[ 2931.,  2494.,  2266.],
         [ 2672.,  1388.,    24.],
         [ 3199.,  7602.,  5672.]],

        [[ -511.,  6304., -2546.],
         [ 1704., -5466.,  2307.],
         [ 5092.,  5691.,  1111.]],

        [[  644., -6511., -1961.],
         [ 4175., -1759., -4617.],
         [-2317.,  4556., -3143.]]]], dtype=float32)

In [5]:
print(dict_rtl["global_out"].shape)
np.transpose(dict_rtl["global_out"], (0,3,1,2))

(1, 3, 3, 4)


array([[[[ 4851., -5781.,  -466.],
         [ -374., -4851.,  -633.],
         [ 2689., -2184.,  -457.]],

        [[ 2931.,  2494.,  2266.],
         [ 2672.,  1388.,    24.],
         [ 3199.,  7602.,  5672.]],

        [[ -511.,  6304., -2546.],
         [ 1704., -5466.,  2307.],
         [ 5092.,  5691.,  1111.]],

        [[  644., -6511., -1961.],
         [ 4175., -1759., -4617.],
         [-2317.,  4556., -3143.]]]], dtype=float32)

In [5]:
SIMD=3
# ((SIMD-1-i) + int'(i/SIMD)*SIMD) + int'(i/SIMD) * SIMD
for i in range(0,12):
    a0 = (SIMD-1-i) + int(i/SIMD)*SIMD
    a1 = int(i/SIMD)*SIMD
    print("{} {} {}".format(a0,a1,a0+a1))
# (SIMD-1-i) % SIMD + (i/SIMD)*SIMD

2 0 2
1 0 1
0 0 0
2 3 5
1 3 4
0 3 3
2 6 8
1 6 7
0 6 6
2 9 11
1 9 10
0 9 9


In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe
from qonnx.util.basic import (
    gen_finn_dt_tensor
)

model = ModelWrapper("partition_rtl_vvau.onnx")
input_dict = {"global_in" : gen_finn_dt_tensor(model.get_tensor_datatype("global_in"), model.get_tensor_shape("global_in"))}

build_dir = "/scratch/mirzam/build_files"
model.set_metadata_prop("rtlsim_trace", build_dir+"/rtl-vvu.vcd")
# set top-level prop for stitched-ip rtlsim and launch
model.set_metadata_prop("exec_mode", "rtlsim")
vvu_rtl_out = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)

> /scratch/mirzam/finn/src/finn/core/onnx_exec.py(95)execute_onnx()
     93         # use stitched IP for rtlsim
     94         import pdb; pdb.set_trace()
---> 95         rtlsim_exec(model, execution_context)
     96     else:
     97         raise Exception(



ipdb>  s


--Call--
> /scratch/mirzam/finn/src/finn/core/rtlsim_exec.py(43)rtlsim_exec()
     41 
     42 
---> 43 def rtlsim_exec(model, execution_context, pre_hook=None, post_hook=None):
     44     """Use PyVerilator to execute given model with stitched IP. The execution
     45     context contains the input values. Hook functions can be optionally



ipdb>  s


> /scratch/mirzam/finn/src/finn/core/rtlsim_exec.py(51)rtlsim_exec()
     49     - post_hook : hook function to be called after sim end
     50     """
---> 51     if PyVerilator is None:
     52         raise ImportError("Installation of PyVerilator is required.")
     53     # ensure stitched ip project already exists



...

*** NameError: name 'rtlsim_so' is not defined


ipdb>  n


> /scratch/mirzam/finn/src/finn/core/rtlsim_exec.py(129)rtlsim_exec()
    127     # prepare pyverilator model
    128     rtlsim_so = model.get_metadata_prop("rtlsim_so")
--> 129     if (rtlsim_so is None) or (not os.path.isfile(rtlsim_so)):
    130         sim = pyverilate_stitched_ip(model, extra_verilator_args=extra_verilator_args)
    131         model.set_metadata_prop("rtlsim_so", sim.lib._name)



ipdb>  rtlsim_so


'/scratch/mirzam/build_files/pyverilator_ipstitched_fdjz8nvu/Vfinn_design_wrapper'


In [22]:
vvu_rtl_out

{'global_in': array([[[[ 113.,  125., -101.,  -40.],
          [ -28.,  102.,   98.,  125.],
          [ -63.,   43.,   66.,   35.],
          [ -75.,  -55.,  -64.,  -62.],
          [  30.,  -83.,   81.,  -95.]],
 
         [[  96.,  -87.,   29., -106.],
          [  39.,  -45.,  -40.,  -60.],
          [  52.,  -44.,  -45.,   84.],
          [  95.,   58.,  126.,  -96.],
          [ -76.,  113.,  120.,  -35.]],
 
         [[ 105., -112., -111.,  -23.],
          [ -94.,  -24.,  -18.,   42.],
          [ -37.,  -84.,    2.,  -73.],
          [  62.,  -13., -100., -101.],
          [ -24.,   -8.,  122.,  -36.]],
 
         [[ -38.,  -92., -106.,   57.],
          [  62.,  116., -124.,  -92.],
          [  31.,  -67., -118.,  -85.],
          [ -79.,  117.,   40.,  -94.],
          [ -61.,  -16.,   22.,  -60.]],
 
         [[-115.,  -89.,   49.,   71.],
          [ 110.,   78., -109.,   78.],
          [ -34.,  -49.,  -95.,  -38.],
          [-114.,   80.,   29.,   72.],
          [ -47

In [8]:
a = 10.673539876937866
a = a+1.1306219100952148
a = a+5.44316291809082
a = a+2.610429048538208
a = a+5.802148818969727
a = a+0.32395219802856445
a = a+0.05273103713989258
a = a+8.063836097717285
a = a+326.1223919391632
a = a+2425.6277453899384
a = a+877.1852259635925
a = a+14960.15063881874

print(2426/a)
print(878/a)
print(14960/a)
print(0.05273103713989258/a*100)

0.13026771814254978
0.047145530308804084
0.8032996963778009
0.0002831472334502824


In [1]:
a = 10.251316547393799
a = a+ 1.1076984405517578
a = a+5.140721082687378
a = a+2.5184714794158936
a = a+ 5.734774589538574
a = a+ 0.3354811668395996
a = a+ 0.0513148307800293
a = a+ 7.986029148101807
a = a+ 327.8631513118744
a = a+ 1600.0939083099365
a = a+ 594.3238687515259
a = a+ 11706.895140647888

print(327/a)
print(1600/a)
print(594/a)
print(11706/a)

0.022927575284550226
0.11218385460330386
0.04164825602147656
0.8207651262414218
